# Notebook #2: Carga: creación de Base de Datos SQL

In [7]:
%load_ext autoreload
%autoreload 2

# Librerías para tratamiento de datos

import pandas as pd
pd.set_option('display.max_columns', None) # Parámetro que modifica la visualización de los DFs

# Librería para el acceso a variables y funciones
import sys
sys.path.append("../")
from src import soporte_funciones as sf #Archivo .py donde encontraremos todas nuestras funciones.
from src import soporte_variables as sv

# Librería para trabajar con bases de datos SQL
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors

# Librería para ignorar avisos
import warnings
warnings.filterwarnings("ignore") # Ignora TODOS los avisos

## Crear tuplas

In [65]:
rutas = ["datos/contenido.csv",
"datos/caracteristicas.csv",
"datos/tipo.csv",
"datos/directores.csv"]

lista_contenido, lista_caracteristicas, lista_tipo, lista_directores = sf.csvs_a_tuplas(rutas)

## Creación de tablas

Primero la base de datos

In [37]:
sf.dbeaver_crear_db("pruebatecnica")

Base de datos pruebatecnica creada con éxito


In [66]:
sf.dbeaver_commit(sf.dbeaver_conexion("pruebatecnica"),sv.query_creacion_directores)

Commit realizado


In [67]:
sf.dbeaver_commit(sf.dbeaver_conexion("pruebatecnica"),sv.query_creacion_tipo_contenido)

Commit realizado


In [68]:
sf.dbeaver_commit(sf.dbeaver_conexion("pruebatecnica"),sv.query_creacion_contenido)

Commit realizado


In [69]:
sf.dbeaver_commit(sf.dbeaver_conexion("pruebatecnica"),sv.query_creacion_caracteristicas)

Commit realizado


## Inserciones

In [70]:
sf.dbeaver_commitmany(sf.dbeaver_conexion("pruebatecnica"),sv.query_inser_directores,lista_directores)

Commit realizado


In [71]:
sf.dbeaver_commitmany(sf.dbeaver_conexion("pruebatecnica"),sv.query_inser_tipo_contenido,lista_tipo)

Commit realizado


In [73]:
sf.dbeaver_commitmany(sf.dbeaver_conexion("pruebatecnica"),sv.query_inser_caracteristicas,lista_caracteristicas)

Commit realizado


In [72]:
sf.dbeaver_commitmany(sf.dbeaver_conexion("pruebatecnica"),sv.query_inser_contenido,lista_contenido)

Commit realizado
